In [1]:
import os
import sys
import numpy as np
import tensorflow as tf
import importlib
import time
import cv2
from tqdm import tqdm
import pickle

sys.path.append("../")
import DatasetsBuilder

import getConfig
config,unparsed=getConfig.get_lfconfig()

import buildTrainNetwork
import det_tools
from inference import *

from common.tf_layer_utils import *
from common.tf_train_utils import get_optimizer, get_piecewise_lr, get_activation_fn
from common.tfvisualizer import log_images, convert_tile_image

# main

In [14]:
# tf.reset_default_graph() # for sure

log_dir = config.log_dir
batch_size = config.batch_size
optim_method = config.optim_method
learning_rate = config.lr
va_batch_size = 1

tr_loader = DatasetsBuilder.SfMDataset(out_size=(362, 362), 
                   warp_aug_mode='random', flip_pair=True, max_degree=180, max_scale=1.414,
                   num_threads=2)
tr_dataset = tr_loader.get_dataset('../dataset', 'config.sfm_img_dir', 
                    'scan', phase='train',
                    batch_size=6, shuffle=True)
# config.depth_thresh = tr_loader.depth_thresh


['../dataset/scan/train.tfrecord']
aug_mode=random max_rad=3.141592653589793, max_scale_log=0.34642256747438094
Add random logscale=-0.35~0.35, ori=-3.141592653589793~3.141592653589793


In [3]:
iterator = tr_dataset.make_one_shot_iterator()
next_element = iterator.get_next()
next_batch = list(next_element)

In [4]:
next_batch = DatasetsBuilder.euclidean_augmentation(next_batch, (config.data_size, config.data_size), config.rot_aug, config.scale_aug)

Tensor("strided_slice_1:0", shape=(?, ?), dtype=float32)


In [5]:
is_training_ph = tf.placeholder(tf.bool, shape=(), name='is_training')
psf = tf.constant(det_tools.get_gauss_filter_weight(config.hm_ksize, config.hm_sigma)[:,:,None,None], dtype=tf.float32) 
global_step = tf.Variable(0, name='global_step', trainable=False)

In [6]:
det_loss, desc_loss, det_endpoints, desc_endpoints, eval_endpoints, sift_endpoints = \
                    buildTrainNetwork.build_training_network(config, next_batch, is_training_ph, psf, global_step)

Apply instance norm on input photos
Act-Fn:  <function get_activation_fn.<locals>.<lambda> at 0x7f91f6bd9510>
Apply 3D NMS instead.
Scales (0.707107~1.41 #5): [1.41421356 1.18920712 1.         0.84089642 0.70710678]
Instructions for updating:
dim is deprecated, use axis instead
INFO:tensorflow:Summary name ConvOnlyResNet/ori_conv/weights:0 is illegal; using ConvOnlyResNet/ori_conv/weights_0 instead.
INFO:tensorflow:Summary name ConvOnlyResNet/ori_conv/biases:0 is illegal; using ConvOnlyResNet/ori_conv/biases_0 instead.
PAD=16, #conv=8, ksize=5 ori-ksize=5
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Scales (0.707107~1.41 #5): [1.41421356 1.18920712 1.         0.84089642 0.70710678]
PAD=16, #conv=8, ksize=5 ori-ksize=5
Act-Fn:  <function relu at 0x7f920da089d8>
===== SimpleDesc (reuse=False) =====
#1 conv-bn-act (?, 16, 16, 64)
#2 conv-bn-act (?, 8, 8, 128)
#3 conv-bn-act (?, 4, 4, 256)


In [9]:
det_var_list = det_endpoints['var_list'] + det_endpoints['mso_var_list']
desc_var_list = desc_endpoints['var_list']

In [15]:
desc_minimize_op = get_optimizer(optim_method, global_step, learning_rate, desc_loss, desc_var_list, show_var_and_grad=config.show_histogram)
# detector minimizer
det_minimize_op = get_optimizer(optim_method, global_step, learning_rate, det_loss, det_var_list, show_var_and_grad=config.show_histogram)

========== get_optimizer (adam) ==========


/home/wang/.conda/envs/py3-tf/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


SimpleDesc/conv1/weights:0 (3, 3, 1, 64)
SimpleDesc/conv1/biases:0 (64,)
SimpleDesc/bn1/gamma:0 (64,)
SimpleDesc/bn1/beta:0 (64,)
SimpleDesc/conv2/weights:0 (3, 3, 64, 128)
SimpleDesc/conv2/biases:0 (128,)
SimpleDesc/bn2/gamma:0 (128,)
SimpleDesc/bn2/beta:0 (128,)
SimpleDesc/conv3/weights:0 (3, 3, 128, 256)
SimpleDesc/conv3/biases:0 (256,)
SimpleDesc/bn3/gamma:0 (256,)
SimpleDesc/bn3/beta:0 (256,)
SimpleDesc/fc1/weights:0 (4096, 512)
SimpleDesc/fc1/biases:0 (512,)
SimpleDesc/fc-bn1/gamma:0 (512,)
SimpleDesc/fc-bn1/beta:0 (512,)
SimpleDesc/fc2/weights:0 (512, 256)
SimpleDesc/fc2/biases:0 (256,)
========== get_optimizer (adam) ==========
ConvOnlyResNet/init_conv/weights:0 (5, 5, 1, 16)
ConvOnlyResNet/init_conv/biases:0 (16,)
ConvOnlyResNet/block-1/pre-bn/gamma:0 (16,)
ConvOnlyResNet/block-1/pre-bn/beta:0 (16,)
ConvOnlyResNet/block-1/conv1/weights:0 (5, 5, 16, 16)
ConvOnlyResNet/block-1/conv1/biases:0 (16,)
ConvOnlyResNet/block-1/mid-bn/gamma:0 (16,)
ConvOnlyResNet/block-1/mid-bn/beta:0 (

In [16]:
tfconfig = tf.ConfigProto()
tfconfig.gpu_options.allow_growth = True # almost the same as tf.InteractiveSession
sess = tf.Session(config=tfconfig)

In [18]:
sess.run(tf.global_variables_initializer())
tr_iter = tr_dataset.make_one_shot_iterator() # infinite loop

tr_handle = sess.run(tr_iter.string_handle())

In [ ]:
# best_saver = tf.train.Saver(max_to_keep=1, save_relative_paths=True)
# # latest_saver = tf.train.Saver(max_to_keep=1, save_relative_paths=True)
# latest_saver = tf.train.Saver(max_to_keep=100, save_relative_paths=True) # save everything
